In [440]:
#----------------------------------------------------------------------------------------------------------------------
#Librerias necesarias para la elaboracion del reporte
#----------------------------------------------------------------------------------------------------------------------

import plotly.offline as py
import plotly.graph_objs as go
import Libreria_Graficos as lg
import datetime as dt
import pandas as pd
import numpy as np
from scipy import stats
import ipywidgets as widgets
from IPython.display import display
import xlsxwriter

In [441]:
py.init_notebook_mode(connected=True)

In [442]:
%%html
<style>
    h1, h3, p {
    text-align: center;
    color: blue;
    }
    table, th, td {
    border: 1px solid black;
    border-collapse: collapse;
    text-align: center;
    padding: 0px
    }
    div.output {
    align-items: center
    }
</style>

# Reporte de tasas
### Diciembre 2016

In [443]:
#----------------------------------------------------------------------------------------------------------------------
#Funcion que crea los graficos de los reportes
#----------------------------------------------------------------------------------------------------------------------

def grafico_3(x1,y1,y2,y3,colores1,colores2,minimo,maximo,ancho,alto,archivo,lineas):
        
    trace1 = go.Scatter(x=x1,
                        y=y1,
                        name='Saldo Abaco',
                        marker=dict(
                                    color=colores1[0]),
                        yaxis='y',
                        visible=lineas[0]
                        )
    trace2 = go.Bar(x=x1,
                        y=y2,
                        name='Saldo Negocios',
                        marker=dict(
                                    color=colores1[1]),
                        yaxis='y',
                        visible=lineas[1]
                        )
    trace3 = go.Scatter(x=x1,
                        y=y3,
                        name='Tasa',
                        marker=dict(
                                    color=colores1[2]),
                        yaxis='y2',
                        visible=lineas[2]
                        )
  
    data = [trace1, trace2, trace3]

    annotation1 = go.Annotation(
                  x='2015-12-31',
                  y=maximo,
                  text='2016',
                  showarrow=False,
                  xanchor= 'left',
                  font=go.Font(size=12)
                  )  
    annotation2 = go.Annotation(
                  x='2014-12-31',
                  y=maximo,
                  text='2015',
                  showarrow=False,
                  xanchor= 'left',
                  font=go.Font(size=12)
                  )  
    annotation_1=[annotation1, annotation2]

    last_day=max(x1)
    first_day=last_day.replace(day=1)
    
    #El bucle va de 0 a 4, creando los 5 periodos con los cuales se crearan las tendencias
  
    k=3
        
    for i in range(0,5):

            if i==0: 
                dias=30
                per=' 1m'
            elif i==1:
                dias=90
                per=' 3m'
            elif i==2:    
                dias=180
                per=' 6m'
            elif i==3:    
                dias=360
                per=' 1y'
            elif i==4:    
                dias=len(x1)
                per=' all'

            last_day=max(x1)
            xi=np.arange(0,dias)
            first_day=last_day-dt.timedelta(days=dias)
            #El bucle va de 0 a 2, creando las tentencias de las 3 series para el periodo especificado
            for j in range(0,3):
                if j==0: 
                   y=y1[(y1.index > first_day)]
                   nombre='T Abaco'+per
                   yaxis='y'
                elif j==1:
                    y=y2[(y2.index > first_day)]
                    nombre='T Negocios'+per
                    yaxis='y'
                elif j==2:    
                    y=y3[(y3.index > first_day)]
                    nombre='T Tasa'+per
                    yaxis='y2'     
                
                slope, intercept, r_value, p_value, std_err = stats.linregress(xi,y)
                line = slope*xi+intercept
    
                tracei=go.Scatter(x=y.index,
                                  y=line,
                                  name=nombre,
                                  mode='lines',
                                  yaxis=yaxis,
                                  hoverinfo='none',
                                  marker=dict(
                                              color=colores2[j]),
                                  visible=lineas[k],
                                  showlegend=False)
                    
                annotationi=go.Annotation(
                                          x =last_day,
                                          y =maximo*.7,
                                          #text='$R^2 = $'+str(round(r_value,2))+'$,\\Y = $'+str(round(slope,2))+'$X + $'+str(round(intercept,2)),
                                          text='R^2 = %s,\n Y = %sX + %s'%(round(r_value,2),round(slope,2),round(intercept,2)),
                                          showarrow=False,
                                          xanchor='right',
                                          visible=lineas[k],
                                          font=go.Font(size=12)
                                          )  
                annotation_1[len(annotation_1):] = [annotationi]    
            
                data[len(data):] = [tracei]
                #print(k)
                k+=1
    
    last_day=max(x1)
    first_day=last_day.replace(day=1)                
    
    layout = dict(
        title=str(archivo),
        xaxis=dict(
            rangeselector=dict(
                buttons=list([
                    dict(count=1,
                         label='1m',
                         step='month',
                         stepmode='backward'),
                    dict(count=3,
                         label='3m',
                         step='month',
                         stepmode='backward'),
                    dict(count=6,
                         label='6m',
                         step='month',
                         stepmode='backward'),
                    dict(count=1,
                        label='YTD',
                        step='year',
                        stepmode='todate'),
                    dict(count=1,
                        label='1y',
                        step='year',
                        stepmode='backward'),
                    dict(step='all')
                ])
            ),
            rangeslider=dict(),
            type='date',
            #Estableciendo el rango de inicio del grafico 
            range=[first_day,last_day]),
                    
        yaxis=dict(title='Volumen en MM',
                   range=[minimo, maximo]),
        yaxis2=dict(title='Tasas',
                    titlefont=dict(color='rgb(148, 103, 189)'),
                    tickfont=dict(color='rgb(148, 103, 189)'),
                    overlaying='y',
                    side='right'),

        #Añadiendo las formas en las fechas de cambio de año

        shapes=[ {
            'type': 'rect',
            'x0': '2015-12-31',
            'y0': 0,
            'x1': '2015-12-31',
            'y1': 10000000000000,
            'line': {
                    'color': 'rgb(128,128,128)',
                    'width': 1,
            }},
                {
            'type': 'rect',
            'x0': '2014-12-31',
            'y0': 0,
            'x1': '2014-12-31',
            'y1': 10000000000000,
            'line': {
                    'color': 'rgb(128,128,128)',
                    'width': 1,
            }}
            ],
             
        #Creando la lista desplegable para las tendencias                                        

        annotations = annotation_1, #[annotation1, annotation2],
        width=ancho,
        height=alto,
        showlegend=True,
        legend=dict(
                    orientation= "h",x=0,y=-0.4) #Borrar si se quiere la legenda de forma vertical
        )

    fig = go.Figure(data=data, layout=layout)
    return fig

#----------------------------------------------------------------------------------------------------------------------
#Funcion que descarga excel con la data y graficos
#----------------------------------------------------------------------------------------------------------------------

def descargar_excel(resultado, nombre):
    
    nro=len(resultado.index)+1
    
    writer = pd.ExcelWriter(str(nombre)+'.xlsx', engine='xlsxwriter')

    resultado.to_excel(writer, sheet_name='Sheet1')

    workbook  = writer.book
    worksheet = writer.sheets['Sheet1']

    bold = workbook.add_format({'bold': 1})

    headings = ['Fecha', 'Saldo Abaco', 'Saldo Negocios', 'Tasa']

    worksheet.write_row('A1', headings, bold)

    # Creando un nuevo objeto grafico
    chart1 = workbook.add_chart({'type': 'line'})

    # Configurando las series que se van a ingresar en el grafico
    chart1.add_series({
        'name':       '=Sheet1!$B$1',
        'categories': '=Sheet1!$A$2:$A$'+str(nro),
        'values':     '=Sheet1!$B$2:$B$'+str(nro),
        'line': {'color': '#1F490C',}, 
    })

    chart1.add_series({
        'name':       '=Sheet1!$C$1',
        'categories': '=Sheet1!$A$2:$A$'+str(nro),
        'values':     '=Sheet1!$C$2:$C$'+str(nro),
        'line': {'color': '#95B3D7',},
    })

    chart1.add_series({
        'name':       '=Sheet1!$D$1',
        'categories': '=Sheet1!$A$2:$A$'+str(nro),
        'values':     '=Sheet1!$D$2:$D$'+str(nro),
        'type':       'column',
        'y2_axis':    1,
         'line': {'color': '#F09646',},   
    })

    # Adicionando el formato del grafico
    chart1.set_size({'width': 800, 'height': 400})
    chart1.set_title ({'name': nombre})
    chart1.set_x_axis({'date_axis':  True,
                       'num_format': 'mmm-yy'})
    chart1.set_y_axis({'name': 'Volumen en MM',
                      'num_format': '0.0,,'})
    chart1.set_y2_axis({'name': 'Tasa',
                        'num_format': '0.00'})

    # Estableciendo el estilo del grafico
    chart1.set_style(1)

    # Insertando el grafico en el excel
    worksheet.insert_chart('D2', chart1, {'x_offset': 100, 'y_offset': 20})

    workbook.close()

In [444]:
#----------------------------------------------------------------------------------------------------------------------
#Importando la data para elaborar los graficos
#----------------------------------------------------------------------------------------------------------------------

df1 = pd.read_excel('Data1.xlsx',index_col=0)
df2 = pd.read_excel('Data2.xlsx',index_col=0)
df3 = pd.read_excel('Data3.xlsx',index_col=0)
df4 = pd.read_excel('Data4.xlsx',index_col=0)

titulos = pd.read_excel('Titulos.xlsx', header=0 )

x = df1.index
a=len(x)

colores1=['rgb(31, 73, 12)', 'rgb(149, 179, 215)','rgb(247, 150, 70)']
colores2=['rgb(31, 73, 12)', 'rgb(149, 179, 215)','rgb(247, 150, 70)']
lineas1= [True, True, True,
          False, False, False, 
          False, False, False, 
          False, False, False, 
          False, False, False, 
          False, False, False]

In [445]:
#----------------------------------------------------------------------------------------------------------------------
#Elaborando el primer grafico
#----------------------------------------------------------------------------------------------------------------------

n=0
nombre=titulos.iloc[n,0]

y1=df1[df1.columns[n]]
y2=df2[df2.columns[n]]

if df3[df3.columns[n]].isnull().sum()==a:
    y3=df4[df4.columns[n]]
else:
    y3=df3[df3.columns[n]]    
    
min1=y1.min()*0.95
max1=y1.max()*1.05

f=grafico_3(x,y1,y2,y3,colores1, colores2, min1, max1, 900, 500, nombre,lineas1)

#----------------------------------------------------------------------------------------------------------------------
#----------------------------------------------------------------------------------------------------------------------

c3=[]
for i in range(0,3):
    c1=f['data']
    c2=c1[i]
    c3[len(c3):]=[c2['y']]

resultado1 = pd.concat([c3[0], c3[1], c3[2]], axis=1)

button = widgets.Button(description="Descargar")

variable={'Saldo Abaco':['Ninguna','1m','3m','6m','1y','All'],'Saldo Negocio':['Ninguna','1m','3m','6m','1y','All'],'Tasa':['Ninguna','1m','3m','6m','1y','All']}
variableWs = {key: widgets.Dropdown(options=variable[key], description='Periodo',flex='2 1 auto', width='auto', height='40%') for key in variable}

def grafico_3v2(**func_kwargs):
    lineas_1= [True, True, True,
          False, False, False, 
          False, False, False, 
          False, False, False, 
          False, False, False, 
          False, False, False]
    if (serieW.value=='Saldo Abaco' and tendenciaW.value=='1m'):
        lineas_1[3]= True
    elif (serieW.value=='Saldo Abaco' and tendenciaW.value=='3m'):
        lineas_1[6]= True
    elif (serieW.value=='Saldo Abaco' and tendenciaW.value=='6m'):
        lineas_1[9]= True
    elif (serieW.value=='Saldo Abaco' and tendenciaW.value=='1y'):
        lineas_1[12]= True
    elif (serieW.value=='Saldo Abaco' and tendenciaW.value=='All'):
        lineas_1[15]= True
    elif (serieW.value=='Saldo Negocios' and tendenciaW.value=='1m'):
        lineas_1[4]= True
    elif (serieW.value=='Saldo Negocios' and tendenciaW.value=='3m'):
        lineas_1[7]= True
    elif (serieW.value=='Saldo Negocios' and tendenciaW.value=='6m'):
        lineas_1[10]= True
    elif (serieW.value=='Saldo Negocios' and tendenciaW.value=='1y'):
        lineas_1[13]= True
    elif (serieW.value=='Saldo Negocios' and tendenciaW.value=='All'):
        lineas_1[16]= True
    elif (serieW.value=='Tasa' and tendenciaW.value=='1m'):
        lineas_1[5]= True
    elif (serieW.value=='Tasa' and tendenciaW.value=='3m'):
        lineas_1[8]= True
    elif (serieW.value=='Tasa' and tendenciaW.value=='6m'):
        lineas_1[11]= True
    elif (serieW.value=='Tasa' and tendenciaW.value=='1y'):
        lineas_1[14]= True
    elif (serieW.value=='Tasa' and tendenciaW.value=='All'):
        lineas_1[17]= True
        
    f=grafico_3(x, y1, y2, y3, colores1, colores2, min1, max1, 900, 500, nombre, lineas_1)
    py.iplot(f, show_link=False)

def select_serie(serie):
    new_i = widgets.interactive(grafico_3v2, serie=serieW, tendencia=variableWs[variable['new']])
    i.children = new_i.children

serieW = widgets.Dropdown(options=list(variable.keys()), description='Serie', flex='2 1 auto', width='auto', height='40%')
init = serieW.value
tendenciaW = variableWs[init]

serieW.observe(select_serie, 'value')

i = widgets.interactive(grafico_3v2, serie=serieW, tendencia=tendenciaW)

container1 = widgets.Accordion([button])
container1.set_title(0,'Descargar Data')

container2 = widgets.Accordion([widgets.HBox([i.children[1],i.children[0]])])
container2.set_title(0,'Calcular Tendencia')

container3 = widgets.HBox([container1, container2])

display(container3)

py.iplot(f, show_link=False)

def on_button_clicked(b):
    descargar_excel(resultado1, nombre)

button.on_click(on_button_clicked)

In [446]:
#----------------------------------------------------------------------------------------------------------------------
#Elaborando el primer grafico
#----------------------------------------------------------------------------------------------------------------------

n=1
nombre=titulos.iloc[n,0]

y1=df1[df1.columns[n]]
y2=df2[df2.columns[n]]

if df3[df3.columns[n]].isnull().sum()==a:
    y3=df4[df4.columns[n]]
else:
    y3=df3[df3.columns[n]]    
    
min1=y1.min()*0.95
max1=y1.max()*1.05

f=grafico_3(x,y1,y2,y3,colores1, colores2, min1, max1, 900, 500, nombre,lineas1)

#----------------------------------------------------------------------------------------------------------------------
#----------------------------------------------------------------------------------------------------------------------

c3=[]
for i in range(0,3):
    c1=f['data']
    c2=c1[i]
    c3[len(c3):]=[c2['y']]

resultado2 = pd.concat([c3[0], c3[1], c3[2]], axis=1)

button = widgets.Button(description="Descargar")

variable={'Saldo Abaco':['Ninguna','1m','3m','6m','1y','All'],'Saldo Negocio':['Ninguna','1m','3m','6m','1y','All'],'Tasa':['Ninguna','1m','3m','6m','1y','All']}
variableWs = {key: widgets.Dropdown(options=variable[key], description='Periodo',flex='2 1 auto', width='auto', height='40%') for key in variable}

def grafico_3v2(**func_kwargs):
    lineas_1= [True, True, True,
          False, False, False, 
          False, False, False, 
          False, False, False, 
          False, False, False, 
          False, False, False]
    if (serieW.value=='Saldo Abaco' and tendenciaW.value=='1m'):
        lineas_1[3]= True
    elif (serieW.value=='Saldo Abaco' and tendenciaW.value=='3m'):
        lineas_1[6]= True
    elif (serieW.value=='Saldo Abaco' and tendenciaW.value=='6m'):
        lineas_1[9]= True
    elif (serieW.value=='Saldo Abaco' and tendenciaW.value=='1y'):
        lineas_1[12]= True
    elif (serieW.value=='Saldo Abaco' and tendenciaW.value=='All'):
        lineas_1[15]= True
    elif (serieW.value=='Saldo Negocios' and tendenciaW.value=='1m'):
        lineas_1[4]= True
    elif (serieW.value=='Saldo Negocios' and tendenciaW.value=='3m'):
        lineas_1[7]= True
    elif (serieW.value=='Saldo Negocios' and tendenciaW.value=='6m'):
        lineas_1[10]= True
    elif (serieW.value=='Saldo Negocios' and tendenciaW.value=='1y'):
        lineas_1[13]= True
    elif (serieW.value=='Saldo Negocios' and tendenciaW.value=='All'):
        lineas_1[16]= True
    elif (serieW.value=='Tasa' and tendenciaW.value=='1m'):
        lineas_1[5]= True
    elif (serieW.value=='Tasa' and tendenciaW.value=='3m'):
        lineas_1[8]= True
    elif (serieW.value=='Tasa' and tendenciaW.value=='6m'):
        lineas_1[11]= True
    elif (serieW.value=='Tasa' and tendenciaW.value=='1y'):
        lineas_1[14]= True
    elif (serieW.value=='Tasa' and tendenciaW.value=='All'):
        lineas_1[17]= True
        
    f=grafico_3(x, y1, y2, y3, colores1, colores2, min1, max1, 900, 500, nombre, lineas_1)
    py.iplot(f, show_link=False)

def select_serie(serie):
    new_i = widgets.interactive(grafico_3v2, serie=serieW, tendencia=variableWs[variable['new']])
    i.children = new_i.children

serieW = widgets.Dropdown(options=list(variable.keys()), description='Serie', flex='2 1 auto', width='auto', height='40%')
init = serieW.value
tendenciaW = variableWs[init]

serieW.observe(select_serie, 'value')

i = widgets.interactive(grafico_3v2, serie=serieW, tendencia=tendenciaW)

container1 = widgets.Accordion([button])
container1.set_title(0,'Descargar Data')

container2 = widgets.Accordion([widgets.HBox([i.children[1],i.children[0]])])
container2.set_title(0,'Calcular Tendencia')

container3 = widgets.HBox([container1, container2])

display(container3)

py.iplot(f, show_link=False)

def on_button_clicked(b):
    descargar_excel(resultado2, nombre)

button.on_click(on_button_clicked)

In [457]:
#----------------------------------------------------------------------------------------------------------------------
#Elaborando el primer grafico
#----------------------------------------------------------------------------------------------------------------------

n=2
nombre=titulos.iloc[n,0]

y1=df1[df1.columns[n]]
y2=df2[df2.columns[n]]

if df3[df3.columns[n]].isnull().sum()==a:
    y3=df4[df4.columns[n]]
else:
    y3=df3[df3.columns[n]]    
    
min1=y1.min()*0.95
max1=y1.max()*1.05

f=grafico_3(x,y1,y2,y3,colores1, colores2, min1, max1, 900, 500, nombre,lineas1)

#----------------------------------------------------------------------------------------------------------------------
#----------------------------------------------------------------------------------------------------------------------

c3=[]
for i in range(0,3):
    c1=f['data']
    c2=c1[i]
    c3[len(c3):]=[c2['y']]

resultado3= pd.concat([c3[0], c3[1], c3[2]], axis=1)

button = widgets.Button(description="Descargar")

variable={'Saldo Abaco':['Ninguna','1m','3m','6m','1y','All'],'Saldo Negocio':['Ninguna','1m','3m','6m','1y','All'],'Tasa':['Ninguna','1m','3m','6m','1y','All']}
variableWs = {key: widgets.Dropdown(options=variable[key], description='Periodo',flex='2 1 auto', width='auto', height='40%') for key in variable}

def grafico_3v2(**func_kwargs):
    lineas_1= [True, True, True,
          False, False, False, 
          False, False, False, 
          False, False, False, 
          False, False, False, 
          False, False, False]
    if (serieW.value=='Saldo Abaco' and tendenciaW.value=='1m'):
        lineas_1[3]= True
    elif (serieW.value=='Saldo Abaco' and tendenciaW.value=='3m'):
        lineas_1[6]= True
    elif (serieW.value=='Saldo Abaco' and tendenciaW.value=='6m'):
        lineas_1[9]= True
    elif (serieW.value=='Saldo Abaco' and tendenciaW.value=='1y'):
        lineas_1[12]= True
    elif (serieW.value=='Saldo Abaco' and tendenciaW.value=='All'):
        lineas_1[15]= True
    elif (serieW.value=='Saldo Negocios' and tendenciaW.value=='1m'):
        lineas_1[4]= True
    elif (serieW.value=='Saldo Negocios' and tendenciaW.value=='3m'):
        lineas_1[7]= True
    elif (serieW.value=='Saldo Negocios' and tendenciaW.value=='6m'):
        lineas_1[10]= True
    elif (serieW.value=='Saldo Negocios' and tendenciaW.value=='1y'):
        lineas_1[13]= True
    elif (serieW.value=='Saldo Negocios' and tendenciaW.value=='All'):
        lineas_1[16]= True
    elif (serieW.value=='Tasa' and tendenciaW.value=='1m'):
        lineas_1[5]= True
    elif (serieW.value=='Tasa' and tendenciaW.value=='3m'):
        lineas_1[8]= True
    elif (serieW.value=='Tasa' and tendenciaW.value=='6m'):
        lineas_1[11]= True
    elif (serieW.value=='Tasa' and tendenciaW.value=='1y'):
        lineas_1[14]= True
    elif (serieW.value=='Tasa' and tendenciaW.value=='All'):
        lineas_1[17]= True
        
    f=grafico_3(x, y1, y2, y3, colores1, colores2, min1, max1, 900, 500, nombre, lineas_1)
    py.iplot(f, show_link=False)

def select_serie(serie):
    new_i = widgets.interactive(grafico_3v2, serie=serieW, tendencia=variableWs[variable['new']])
    i.children = new_i.children

serieW = widgets.Dropdown(options=list(variable.keys()), description='Serie', flex='2 1 auto', width='auto', height='40%')
init = serieW.value
tendenciaW = variableWs[init]

serieW.observe(select_serie, 'value')

i = widgets.interactive(grafico_3v2, serie=serieW, tendencia=tendenciaW)

container1 = widgets.Accordion([button], visible=False)
container1.set_title(0,'Descargar Data')

container2 = widgets.Accordion([widgets.HBox([i.children[1],i.children[0]])])
container2.set_title(0,'Calcular Tendencia')

container3 = widgets.HBox([container1, container2])

display(container3)

py.iplot(f, show_link=False)

def on_button_clicked(b):
    descargar_excel(resultado3, nombre)

button.on_click(on_button_clicked)